# Import

In [1]:
import pandas as pd
import pickle
import os

# Load data
## Loading

In [2]:
n_carriers_per_node = 30
node_auction_cost = 0.

In [3]:
path = os.path.join(os.getcwd(), 'Data_analysis')

carriers_file_name = 'carriers_df_' + str(node_auction_cost) + '_' + str(n_carriers_per_node) + '.bin'
loads_file_name = 'loads_df_' + str(node_auction_cost) + '_' + str(n_carriers_per_node) + '.bin'
movements_file_name = 'movements_df_' + str(node_auction_cost) + '_' + str(n_carriers_per_node) + '.bin'

with open(os.path.join(path, carriers_file_name), 'rb') as f:
    carriers_df = pickle.load(f)
with open(os.path.join(path, loads_file_name), 'rb') as f:
    loads_df = pickle.load(f)
with open(os.path.join(path, movements_file_name), 'rb') as f:
    movements_df = pickle.load(f)
    
carriers_df['name'] = carriers_df.name.apply(lambda x: x.split('_')[-1]).astype('int64')
carriers_df.set_index(keys=['run_id', 'home', 'name'], inplace=True)
carriers_df.sort_index(inplace=True)
loads_df.set_index(keys=['run_id', 'load_id'], inplace=True)
loads_df.sort_index(inplace=True)
movements_df.set_index(keys=['run_id', 'load_id', 'step'], inplace=True)
movements_df.sort_index(inplace=True)

In [4]:
path = os.path.join(os.getcwd(), '../PI_RPS/Games/data')
distances = pd.read_csv(os.path.join(path, 'city_distance_matrix_time_step.csv'), index_col=0)

## Preparing data

In [32]:
loads_df['direct_distance'] = loads_df.apply(lambda x: distances.loc[x.departure, x.arrival], axis=1)
movements_df['distance'] = movements_df.apply(lambda x: distances.loc[x.origin, x.destination], axis=1)

movements_df['carrier_home'] = movements_df.carrier.apply(lambda x: x.split('_')[0])

movements_df_grouped = movements_df.groupby(['run_id', 'load_id'])

loads_df['route'] = movements_df_grouped['destination'].agg(list).apply(lambda x: '-'.join(x))
loads_df['route'] = loads_df.departure + '-' + loads_df.route

loads_df['nb_hops'] = movements_df_grouped.count().origin.rename('nb_hops')
loads_df['nb_hops'].fillna(-1, inplace=True)
loads_df['nb_hops'] = loads_df['nb_hops'].astype('int64')

loads_df['total_distance'] = movements_df_grouped['distance'].sum()
loads_df['total_distance'].fillna(-1, inplace=True)
loads_df['total_distance'] = loads_df['total_distance'].astype('int64')

loads_df['total_price'] = movements_df_grouped['price'].sum()

loads_df['nb_reserve_price_involved'] = movements_df.reserve_price_involved.astype('int64').groupby(['run_id', 'load_id']).sum()
loads_df['nb_reserve_price_involved'].fillna(-1, inplace=True)
loads_df['nb_reserve_price_involved'] = loads_df['nb_reserve_price_involved'].astype('int64')

loads_df_filtered = loads_df.loc[(loads_df.nb_reserve_price_involved == 0) & (loads_df.direct_distance <= loads_df.total_distance)]

# Analysis
## Sanity check for involvement of reserve price
We want lower than 0.5%

In [10]:
rp_involved_in_mvmts_count = movements_df.groupby('reserve_price_involved').count()
perc_rp_involved_in_mvmts = rp_involved_in_mvmts_count.loc[True, 'origin'] / rp_involved_in_mvmts_count.origin.sum()
print("Reserve price is involved in {}% of the movements".format(round(perc_rp_involved_in_mvmts*100, 1)))

rp_involved_in_loads = 1-(loads_df_filtered.departure.count() / loads_df.loc[(loads_df.nb_reserve_price_involved >= 0) & (loads_df.direct_distance <= loads_df.total_distance)].departure.count())
print("Reserve price is involved in {}% of the loads".format(round(rp_involved_in_loads*100, 1)))

Reserve price is involved in 0.1% of the movements
Reserve price is involved in 0.1% of the loads


## Carriers
### Concentration at nodes

In [34]:
nb_carriers = carriers_df['t_c'].count()
carriers_concentration = carriers_df.groupby('next_node')['t_c'].count()
carriers_concentration_prop = carriers_concentration / nb_carriers
carriers_concentration_origin = carriers_df.pivot_table(index='next_node', columns='home', values='t_c', aggfunc='count', fill_value=0)
carriers_concentration_origin_prop = carriers_concentration_origin / carriers_concentration_origin.sum(axis=1)

print(carriers_concentration_prop)
print(carriers_concentration_origin_prop)

next_node
Bremen         0.082121
Dresden        0.103939
Madrid         0.089697
Marseille      0.085152
Milan          0.107576
Naples         0.090000
Paris          0.111818
Rotterdam      0.059394
Saarbrücken    0.098788
Salzburg       0.080000
Warsaw         0.091515
Name: t_c, dtype: float64
home           Bremen   Dresden    Madrid  Marseille     Milan    Naples  \
next_node                                                                  
Bremen       0.095941  0.201166  0.000000   0.003559  0.022535  0.000000   
Dresden      0.291513  0.055394  0.000000   0.000000  0.014085  0.000000   
Madrid       0.000000  0.002915  0.736486   0.245552  0.000000  0.006734   
Marseille    0.007380  0.002915  0.277027   0.195730  0.078873  0.013468   
Milan        0.007380  0.014577  0.000000   0.170819  0.076056  0.521886   
Naples       0.003690  0.000000  0.000000   0.017794  0.425352  0.430976   
Paris        0.055351  0.037901  0.000000   0.423488  0.008451  0.006734   
Rotterdam    0.2

I am not sure whether or not there are a lot of things to understand from this, just that extreme nodes tend to keep their carriers at home while it is easier for the other to have carriers waiting abroad.

I would expect that extreme nodes can make less profit because of less operations

### Profit and who operate
#### Tables

In [50]:
general_profit = carriers_df.profit / 500

profit_home = carriers_df.groupby('home')['profit'].describe() / 500
profit_home['count'] = 500*profit_home['count']

general_operations = movements_df.groupby(['carrier', 'run_id'])['origin'].count()
#add 0 for the others
operations_home = movements_df.groupby(['carrier', 'run_id'])[['carrier_home']].first()
operations_home['count'] = general_operations
operations_home = operations_home.groupby('carrier_home').describe()

#profit per operation
#profit per operation_home

print(general_profit.describe())
print(profit_home)
print(general_operations.describe())
print(operations_home)

count    3300.000000
mean      121.475606
std        48.948217
min         0.000000
25%       112.637157
50%       138.111769
75%       149.464100
max       256.145890
Name: profit, dtype: float64
             count        mean        std         min         25%         50%  \
home                                                                            
Bremen       300.0  151.684981  20.999890  106.670780  137.200979  146.498289   
Dresden      300.0  147.453420  14.660949  120.211933  139.261421  144.224359   
Madrid       300.0   51.643435  59.092877    0.000000    0.000000   22.997584   
Marseille    300.0  123.014671  33.883063   25.312968  109.667111  132.513295   
Milan        300.0  136.270181  15.649447   62.118868  128.352275  138.310973   
Naples       300.0   95.958680  46.541513    0.000000   59.780201  103.883791   
Paris        300.0  135.339162  21.456040   67.142479  122.267468  135.733217   
Rotterdam    300.0  149.118568  16.282586  118.697163  138.922584  145.189

#### Graphs

In [51]:
# same as before but replace home by costs

## Loads

In [13]:
price_per_total_distance = loads_df_filtered.groupby('total_distance')['total_price'].agg(['mean', 'count']).rename({'mean': 'total_price', 'count': 'nb_info'}, axis=1).reset_index()
price_per_total_distance['price_per_distance'] = price_per_total_distance.total_price / price_per_total_distance.total_distance

price_per_direct_distance = loads_df_filtered.groupby('direct_distance')['total_price'].agg(['mean', 'count']).rename({'mean': 'total_price', 'count': 'nb_info'}, axis=1).reset_index()
price_per_direct_distance['price_per_distance'] = price_per_direct_distance.total_price / price_per_direct_distance.direct_distance

print(price_per_total_distance)
print(price_per_direct_distance)

   total_distance  total_price  nb_info  price_per_distance
0               1   241.961026    39964          241.961026
1               2   436.908262     8570          218.454131
2               3   553.561145     2100          184.520382
3               4   847.983041     1335          211.995760
4               5  1067.089052      596          213.417810
5               6   920.378443       96          153.396407
6               7  1079.880958       51          154.268708
7               8   997.754555        9          124.719319
8               9   903.167861        4          100.351985
   direct_distance  total_price  nb_info  price_per_distance
0                1   242.478252    41630          242.478252
1                2   471.431433     8046          235.715716
2                3   664.402901     1276          221.467634
3                4   921.878835     1194          230.469709
4                5  1137.762239      548          227.552448
5                7  1325.022515   

We see that price per total distance can be very low, this is because of loads being on trucks going home for almost free (see (3,312)).

However, the price per direct distance goes down but not much This is because they have more oportunities to take advantage of the previously described phenomenon (see (4,1984)).

In [14]:
nb_hops_per_direct_distance = loads_df_filtered.groupby('direct_distance')['nb_hops'].describe()

print(nb_hops_per_direct_distance)

                   count      mean       std  min  25%  50%  75%  max
direct_distance                                                      
1                41630.0  1.040740  0.201778  1.0  1.0  1.0  1.0  4.0
2                 8046.0  1.952026  0.486420  1.0  2.0  2.0  2.0  5.0
3                 1276.0  2.492947  0.588075  1.0  2.0  2.0  3.0  5.0
4                 1194.0  3.064489  0.448556  1.0  3.0  3.0  3.0  6.0
5                  548.0  3.978102  0.494462  1.0  4.0  4.0  4.0  6.0
7                   31.0  4.870968  1.231312  1.0  4.5  5.0  6.0  6.0


Looks like it works well in doing the routing

In [20]:
most_used_routes = loads_df_filtered.groupby('route')['total_price'].agg(['count', 'mean']).sort_values(by='count', ascending=False).rename({'mean': 'mean_price'}, axis=1)
most_used_complex_routes = loads_df_filtered.loc[loads_df_filtered.nb_hops > 1].groupby('route')['total_price'].agg(['count', 'mean']).sort_values(by='count', ascending=False).rename({'mean': 'mean_price'}, axis=1)
perce_complex_routes = most_used_complex_routes['count'].sum() / most_used_routes['count'].sum()

print("{} % of the routes are complex routes".format(round(perce_complex_routes*100, 1)))

22.1 % of the routes are complex routes


In [21]:
price_per_departure_destination = loads_df_filtered.groupby(['departure', 'arrival'])['total_price'].describe()
hops_per_departure_destination = loads_df_filtered.groupby(['departure', 'arrival'])['nb_hops'].describe()

print(price_per_departure_destination)
print(hops_per_departure_destination)

                        count         mean         std          min  \
departure arrival                                                     
Bremen    Dresden      1924.0   249.742662   60.920671     0.000000   
          Madrid        206.0  1201.872372  169.584151   562.699648   
          Marseille      88.0   712.570901  130.922105   365.722159   
          Milan         458.0   505.825901   85.546953   176.569084   
          Naples         17.0   668.342688  127.503765   430.964941   
...                       ...          ...         ...          ...   
Warsaw    Naples          1.0  1007.683030         NaN  1007.683030   
          Paris          12.0   732.816519   98.211882   562.821759   
          Rotterdam      27.0   408.066227   65.437961   249.149816   
          Saarbrücken    54.0   413.842284   69.305469   218.032532   
          Salzburg      213.0   419.518209   77.240981   168.652049   

                               25%          50%          75%          max  
